# 混合检索

混合检索同时利用稀疏向量的关键词精确匹配能力和密集向量的语义理解能力，以克服单一向量检索的局限性。

## 稀疏向量 vs 密集向量

### 稀疏向量

稀疏向量采用精准的“词袋”匹配模型，将文档视为一堆词的集合，不考虑其顺序和语法，其中向量的每一个维度都直接对应一个具体的词，非零值则代表该词在文档中的重要性（权重）。

这种方法的优点是可解释性极强（每个维度都代表一个确切的词），无需训练，能够实现关键词的精确匹配。

主要缺点是无法理解语义，例如它无法识别“汽车”和“轿车”是同义词，存在“词汇鸿沟”。

稀疏向量的核心思想是只存储非零值。

例如，一个包含5万个词的词汇表中，“西红柿”在第88位，“炒”在第666位，“蛋”在第999位，它们的BM25权重分别是1.2、0.8、1.5。

可表示为：

- 键值对：将非零元素的索引作为键，值作为值。

In [ ]:
{
    "88": 1.2,
    "666": 0.8,
    "999": 1.5
}


{'88': 1.2, '666': 0.8, '999': 1.5}

- 坐标列表：用一个元组 (维度, [索引列表], [值列表]) 来表示。

In [ ]:
(50000, [88, 666, 999], [1.2, 0.8, 1.5])  # 在 SciPy 等科学计算库中非常常见


(50000, [88, 666, 999], [1.2, 0.8, 1.5])

尽管这两种格式都清晰地记录了文档的关键信息。

但当我们搜索“番茄炒鸡蛋”时，由于“番茄”和“西红柿”是不同的词条（索引不同），模型将无法理解它们的语义相似性。

### 密集向量

也常被称为“语义向量”，是通过深度学习模型学习到的数据（如文本、图像）的低维、稠密的浮点数表示。

主要优点是能够理解同义词、近义词和上下文关系，泛化能力强，在语义搜索任务中表现卓越。缺点是可解释性差。

与稀疏向量不同，密集向量的所有维度都有值，因此使用数组 [] 来表示是最直接的方式。

一个预训练好的语义模型在读取“西红柿炒蛋”后，会输出一个低维的密集向量：

In [ ]:
[0.89, -0.12, 0.77, ..., -0.45] # 一个低维（比如1024维）的浮点数向量


[0.89, -0.12, 0.77, Ellipsis, -0.45]

这个向量本身难以解读，但它在语义空间中的位置可能与“番茄鸡蛋面”、“洋葱炒鸡蛋”等菜肴的向量非常接近。

因为模型理解了它们共享“鸡蛋类菜肴”、“家常菜”、“酸甜口味”等核心概念。

因此，当我们搜索“蛋白质丰富的家常菜”时，即使查询中没有出现任何原文关键词，密集向量也很有可能成功匹配到这份菜谱。

## 混合检索

混合检索主要是解决单一检索技术的局限性。

例如，关键词检索无法理解语义，而向量检索则可能忽略掉必须精确匹配的关键词。

同时利用稀疏向量的精确性和密集向量的泛化性，以应对复杂多变的搜索需求。

### 技术原理与融合方法

混合检索通常并行执行两种检索算法，然后将两组异构的结果集融合成一个统一的排序列表。

以下是两种主流的融合策略：

#### 倒数排序融合 (Reciprocal Rank Fusion, RRF)

RRF 考虑的是每个文档在各自结果集中的排名。

具体来说，一个文档在不同检索系统中的排名越靠前，它的最终得分就越高。

公式为：

$$
RRF_{score}(d) = \sum_{i=1}^{k} \frac{1}{rank_i(d) + c}
$$

- $d$: 待评分的文档。

- $k$：使用的检索系统的数量（这里是2，即稀疏向量检索和密集向量检索）

- $rank_i(d)$：文档 $d$ 在第 $i$ 个检索系统中的排名。

- $c$：一个小的常量（通常取 60），用于降低排名靠后文档的权重，避免它们对结果产生过大影响。

#### 加权线性组合

先将不同检索系统的得分进行归一化（例如，统一到 0-1 区间），然后通过一个权重参数 α 来进行线性组合。

$$ Hybrid_{score} = \alpha \cdot Dense_{score} + (1 - \alpha) \cdot Sparse_{score} $$

通过调整 α 的值，可以灵活地控制语义相似性与关键词匹配在最终排序中的贡献比例。

例如，在电商搜索中，可以调高关键词的权重；而在智能问答中，则可以侧重于语义。

### 混合检索的优势和局限

**优势：**

- 召回率与准确率高：能同时捕获关键词和语义，显著优于单一检索。	

- 灵活性强：可通过融合策略和权重调整，适应不同业务场景。

- 容错性好：关键词检索可部分弥补向量模型对拼写错误或罕见词的敏感性。

**局限：**

- 计算资源消耗大：需要同时维护和查询两套索引。

- 参数调试复杂：融合权重等超参数需要反复实验调优。

- 可解释性仍是挑战：融合后的结果排序理由难以直观分析。

## 实践


In [ ]:
# 连接 Milvus 数据库
from pymilvus import connections

MILVUS_URI = "http://localhost:19530" 

print(f"--> 正在连接到 Milvus: {MILVUS_URI}")
connections.connect(uri=MILVUS_URI)


--> 正在连接到 Milvus: http://localhost:19530


In [ ]:
# 初始化嵌入模型
from pymilvus.model.hybrid import BGEM3EmbeddingFunction

print("--> 正在初始化 BGE-M3 嵌入模型...")
ef = BGEM3EmbeddingFunction(
    model_name="models/bge/bge-m3",  # 指向你下载的文件夹
    device="cpu",
    use_fp16=False
)
print(f"--> 嵌入模型初始化完成。密集向量维度: {ef.dim['dense']}")


--> 正在初始化 BGE-M3 嵌入模型...
--> 嵌入模型初始化完成。密集向量维度: 1024


In [ ]:
# 创建 Collection

from pymilvus import MilvusClient, CollectionSchema, FieldSchema, DataType, Collection

COLLECTION_NAME = "dragon_hybrid_demo"

# 将旧的 Collection 删除
milvus_client = MilvusClient(uri=MILVUS_URI)
if milvus_client.has_collection(COLLECTION_NAME):
    print(f"--> 正在删除已存在的 Collection '{COLLECTION_NAME}'...")
    milvus_client.drop_collection(COLLECTION_NAME)
    
# 构建schema
fields = [
    # 自动生成唯一标识，避免主键冲突
    FieldSchema(name="pk", dtype=DataType.VARCHAR, is_primary=True, auto_id=True, max_length=100),
    # 7个VARCHAR字段用于存储元数据
    FieldSchema(name="img_id", dtype=DataType.VARCHAR, max_length=100),
    FieldSchema(name="path", dtype=DataType.VARCHAR, max_length=256),
    FieldSchema(name="title", dtype=DataType.VARCHAR, max_length=256),
    FieldSchema(name="description", dtype=DataType.VARCHAR, max_length=4096),
    FieldSchema(name="category", dtype=DataType.VARCHAR, max_length=64),
    FieldSchema(name="location", dtype=DataType.VARCHAR, max_length=128),
    FieldSchema(name="environment", dtype=DataType.VARCHAR, max_length=64),
    # 2个VECTOR字段用于存储向量表示
    FieldSchema(name="sparse_vector", dtype=DataType.SPARSE_FLOAT_VECTOR),
    FieldSchema(name="dense_vector", dtype=DataType.FLOAT_VECTOR, dim=ef.dim["dense"])
]

# 如果集合不存在，则创建它及索引
if not milvus_client.has_collection(COLLECTION_NAME):
    print(f"--> 正在创建 Collection '{COLLECTION_NAME}'...")
    schema = CollectionSchema(fields, description="关于龙的混合检索示例")
    # 创建集合
    collection = Collection(name=COLLECTION_NAME, schema=schema, consistency_level="Strong")
    print("--> Collection 创建成功。")

    # 创建索引
    print("--> 正在为新集合创建索引...")
    sparse_index = {"index_type": "SPARSE_INVERTED_INDEX", "metric_type": "IP"}
    collection.create_index("sparse_vector", sparse_index)
    print("稀疏向量索引创建成功。")

    dense_index = {"index_type": "AUTOINDEX", "metric_type": "IP"}
    collection.create_index("dense_vector", dense_index)
    print("密集向量索引创建成功。")

# 加载集合到内存
collection = Collection(COLLECTION_NAME)
collection.load()
print(f"--> Collection '{COLLECTION_NAME}' 已加载到内存。")


--> 正在删除已存在的 Collection 'dragon_hybrid_demo'...
--> 正在创建 Collection 'dragon_hybrid_demo'...
--> Collection 创建成功。
--> 正在为新集合创建索引...
稀疏向量索引创建成功。
密集向量索引创建成功。
--> Collection 'dragon_hybrid_demo' 已加载到内存。


In [ ]:
# 加载数据
import json

DATA_PATH = "data/dragon.json"

# 通过 is_empty 检查避免重复插入。
if collection.is_empty:
    print(f"--> Collection 为空，开始插入数据...")
    with open(DATA_PATH, 'r', encoding='utf-8') as f:
        dataset = json.load(f)

    docs, metadata = [], []
    for item in dataset:
        parts = [
            item.get('title', ''),
            item.get('description', ''),
            item.get('location', ''),
            item.get('environment', ''),
        ]
        docs.append(' '.join(filter(None, parts)))
        metadata.append(item)


--> Collection 为空，开始插入数据...


In [ ]:
# 生成向量

print("--> 正在生成向量嵌入...")
embeddings = ef(docs)
print("--> 向量生成完成。")

# 获取两种向量
sparse_vectors = embeddings["sparse"]    # 稀疏向量：词频统计
dense_vectors = embeddings["dense"]      # 密集向量：语义编码


You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


--> 正在生成向量嵌入...
--> 向量生成完成。


In [ ]:
# 将数据插入 Collection 中

# 为每个字段准备批量数据
img_ids = [doc["img_id"] for doc in metadata]
paths = [doc["path"] for doc in metadata]
titles = [doc["title"] for doc in metadata]
descriptions = [doc["description"] for doc in metadata]
categories = [doc["category"] for doc in metadata]
locations = [doc["location"] for doc in metadata]
environments = [doc["environment"] for doc in metadata]

# 插入数据
#  严格按照 Schema 定义的字段顺序插入，9个字段（7个标量+2个向量）
collection.insert([
    img_ids, paths, titles, descriptions, categories, locations, environments,
    sparse_vectors, dense_vectors
])
# 强制将内存缓冲区数据写入磁盘，使数据立即可搜索
collection.flush()


In [ ]:
# 生成 query 向量

search_query = "悬崖上的巨龙"
search_filter = 'category in ["western_dragon", "chinese_dragon", "movie_character"]'
top_k = 5

print(f"\n{'='*20} 开始混合搜索 {'='*20}")
print(f"查询: '{search_query}'")
print(f"过滤器: '{search_filter}'")

# 生成查询向量
query_embeddings = ef([search_query])
dense_vec = query_embeddings["dense"][0]
sparse_vec = query_embeddings["sparse"]._getrow(0)



==================== 开始混合搜索 ====================
查询: '悬崖上的巨龙'
过滤器: 'category in ["western_dragon", "chinese_dragon", "movie_character"]'


In [ ]:
# 分别进行单独检索

# 定义搜索参数
search_params = {"metric_type": "IP", "params": {}}

# 先执行单独的搜索
print("\n--- [单独] 密集向量搜索结果 ---")
dense_results = collection.search(
    [dense_vec],
    anns_field="dense_vector",
    param=search_params,
    limit=top_k,
    expr=search_filter,
    output_fields=["title", "path", "description", "category", "location", "environment"]
)[0]

for i, hit in enumerate(dense_results):
    print(f"{i+1}. {hit.entity.get('title')} (Score: {hit.distance:.4f})")
    print(f"    路径: {hit.entity.get('path')}")
    print(f"    描述: {hit.entity.get('description')[:100]}...")

print("\n--- [单独] 稀疏向量搜索结果 ---")
sparse_results = collection.search(
    [sparse_vec],
    anns_field="sparse_vector",
    param=search_params,
    limit=top_k,
    expr=search_filter,
    output_fields=["title", "path", "description", "category", "location", "environment"]
)[0]

for i, hit in enumerate(sparse_results):
    print(f"{i+1}. {hit.entity.get('title')} (Score: {hit.distance:.4f})")
    print(f"    路径: {hit.entity.get('path')}")
    print(f"    描述: {hit.entity.get('description')[:100]}...")



--- [单独] 密集向量搜索结果 ---
1. 悬崖上的白龙 (Score: 0.7214)
    路径: data/C3/dragon/dragon02.png
    描述: 一头雄伟的白色巨龙栖息在悬崖边缘，背景是金色的云霞和远方的海岸。它拥有巨大的翅膀和优雅的身姿，是典型的西方奇幻生物。...
2. 中华金龙 (Score: 0.5353)
    路径: data/C3/dragon/dragon06.png
    描述: 一条金色的中华龙在祥云间盘旋，它身形矫健，龙须飘逸，展现了东方神话中龙的威严与神圣。...
3. 驯龙高手：无牙仔 (Score: 0.5231)
    路径: data/C3/dragon/dragon05.png
    描述: 在电影《驯龙高手》中，主角小嗝嗝骑着他的龙伙伴无牙仔在高空飞翔。他们飞向灿烂的太阳，下方是岛屿和海洋，画面充满了冒险与友谊。...

--- [单独] 稀疏向量搜索结果 ---
1. 悬崖上的白龙 (Score: 0.2254)
    路径: data/C3/dragon/dragon02.png
    描述: 一头雄伟的白色巨龙栖息在悬崖边缘，背景是金色的云霞和远方的海岸。它拥有巨大的翅膀和优雅的身姿，是典型的西方奇幻生物。...
2. 中华金龙 (Score: 0.0857)
    路径: data/C3/dragon/dragon06.png
    描述: 一条金色的中华龙在祥云间盘旋，它身形矫健，龙须飘逸，展现了东方神话中龙的威严与神圣。...
3. 驯龙高手：无牙仔 (Score: 0.0639)
    路径: data/C3/dragon/dragon05.png
    描述: 在电影《驯龙高手》中，主角小嗝嗝骑着他的龙伙伴无牙仔在高空飞翔。他们飞向灿烂的太阳，下方是岛屿和海洋，画面充满了冒险与友谊。...


In [43]:
# 使用 RRF 进行混合检索
from pymilvus import RRFRanker, AnnSearchRequest

rerank = RRFRanker(k=60)

# 创建搜索请求
dense_req = AnnSearchRequest([dense_vec], "dense_vector", search_params, limit=top_k)
sparse_req = AnnSearchRequest([sparse_vec], "sparse_vector", search_params, limit=top_k)

# 执行混合搜索
results = collection.hybrid_search(
    [sparse_req, dense_req],
    rerank=rerank,
    limit=top_k,
    output_fields=["title", "path", "description", "category", "location", "environment"]
)[0]

# 打印最终结果
for i, hit in enumerate(results):
    print(f"{i+1}. {hit.entity.get('title')} (Score: {hit.distance:.4f})")
    print(f"    路径: {hit.entity.get('path')}")
    print(f"    描述: {hit.entity.get('description')[:100]}...")


1. 悬崖上的白龙 (Score: 0.0328)
    路径: data/C3/dragon/dragon02.png
    描述: 一头雄伟的白色巨龙栖息在悬崖边缘，背景是金色的云霞和远方的海岸。它拥有巨大的翅膀和优雅的身姿，是典型的西方奇幻生物。...
2. 中华金龙 (Score: 0.0320)
    路径: data/C3/dragon/dragon06.png
    描述: 一条金色的中华龙在祥云间盘旋，它身形矫健，龙须飘逸，展现了东方神话中龙的威严与神圣。...
3. 奔跑的奶龙 (Score: 0.0315)
    路径: data/C3/dragon/dragon04.png
    描述: 一只Q版的黄色小恐龙，有着大大的绿色眼睛和友善的微笑。是一部动画中的角色，非常可爱。...
4. 驯龙高手：无牙仔 (Score: 0.0313)
    路径: data/C3/dragon/dragon05.png
    描述: 在电影《驯龙高手》中，主角小嗝嗝骑着他的龙伙伴无牙仔在高空飞翔。他们飞向灿烂的太阳，下方是岛屿和海洋，画面充满了冒险与友谊。...
5. 霸王龙的怒吼 (Score: 0.0312)
    路径: data/C3/dragon/dragon03.png
    描述: 史前时代的霸王龙张开血盆大口，发出震天的怒吼。在它身后，几只翼龙在阴沉的天空中盘旋，展现了白垩纪的原始力量。...
